In [1]:
from hdmf_ai import ResultsTable  # NOTE: because hdmf_ai modifies the hdmf common namespace, it is important that hdmf_ai is imported before pynwb
from pynwb import NWBHDF5IO

# read an NWB file from local disk
filepath = "/Users/rly/Documents/NWB_Data/dandisets/000409/sub-CSHL047/sub-CSHL047_ses-b52182e7-39f6-4914-9717-136db589706e_behavior+ecephys+image.nwb"
io = NWBHDF5IO(filepath, "r")
nwbfile = io.read()

# the NWB Units table stores information about the sorted single units (putative neurons) after preprocessing
# and spike sorting. each row represents a single unit. this dataset includes many metadata fields (table columns) for
# each unit.
units = nwbfile.units

# the Units table can be most readily viewed as a pandas DataFrame
units_df = units.to_dataframe()
print(units_df)
print(f"There are {len(units_df)} units in this dataset.")

/Users/rly/mambaforge/envs/hdmf-ai/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.1 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/Users/rly/mambaforge/envs/hdmf-ai/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.5.0 because version 2.6.0-alpha is already loaded.
  return func(args[0], **pargs)
/Users/rly/mambaforge/envs/hdmf-ai/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.2.0 because version 0.5.0 is already loaded.
  return func(args[0], **pargs)


                                           spike_times unit_name  \
id                                                                 
0    [0.019766666666666665, 0.028533333333333334, 0...        12   
1    [0.019766666666666665, 0.07186666666666666, 0....       424   
2    [0.020266666666666665, 0.04423333333333333, 0....        78   
3    [0.020766666666666666, 0.023266666666666668, 0...       105   
4    [0.022333333333333334, 0.1083, 0.1305, 0.13916...       110   
..                                                 ...       ...   
553  [1422.7791666666667, 2570.4848666666667, 2575....       543   
554  [1448.522, 1477.2797666666668, 1522.4593666666...       291   
555  [1481.3955333333333, 1573.5977333333333, 1575....       515   
556  [1527.9567333333334, 1750.9875666666667, 2421....       246   
557  [2161.375666666667, 2454.7655666666665, 2710.4...       422   

     presence_ratio_standard_deviation  contamination  noise_cutoff  \
id                                          

In [2]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

# run a simple classifier on the units data to predict the location (brain area) of the unit based on
# the amplitude, firing rate, spike count, and presence ratio. there are several ways to label the brain
# area of a unit. here, we use the label using the coarsest atlas, the Cosmos atlas, which has a total
# of 12 annotation regions. in this dataset, there are 5 unique Cosmos locations.
enc = LabelEncoder()
y = np.uint(enc.fit_transform(units["cosmos_location"].data[:]))
unique_labels = enc.classes_
print(f"There are {len(unique_labels)} unique Cosmos locations in this dataset.")

# split the data into training and testing sets
proportion_train = 0.7
n_train_samples = int(np.round(proportion_train * len(units_df)))
n_test_samples = len(units) - n_train_samples
# train = 0, validate = 1, test = 2
tvt = np.array([0] * n_train_samples + [2] * n_test_samples)
np.random.shuffle(tvt)

feature_names = [
    "median_amplitude",
    "standard_deviation_amplitude",
    "firing_rate",
    "spike_count",
    "presence_ratio",
]
X = units_df[feature_names]
X_train = X[tvt == 0]
y_train = y[tvt == 0]
X_test = X[tvt == 2]
y_test = y[tvt == 2]

# run logistic regression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)
predictions_all = logreg.predict(X)
prediction_proba_all = logreg.predict_proba(X)
prediction_log_proba_all = logreg.predict_log_proba(X)
score = logreg.score(X_test, y_test)
print(f"The logistic regression achieved a score of {score} on the test set!")

There are 5 unique Cosmos locations in this dataset.
The logistic regression achieved a score of 0.47904191616766467 on the test set!


/Users/rly/mambaforge/envs/hdmf-ai/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [3]:
results_table = ResultsTable(
    name="logistic_regression_results",
    description="Results of a simple logisitic regression on the units table",
    n_samples=len(units),
    # not the actual model, just a placeholder for demonstration purposes
    pre_trained_model=["Bloom v1.3"],
)
results_table.add_tvt_split(tvt)
# use the text labels which will become an EnumData with uint encoding
results_table.add_true_label(y)
results_table.add_predicted_probability(prediction_proba_all)
results_table.add_predicted_class(predictions_all)
results_table.add_samples(data=np.arange(len(X)), description="all the samples", table=units)

# add a custom column to the results table to store the log probabilities
results_table.add_column(
    name="log_probability",
    data=prediction_log_proba_all,
    description="The log probability of each class prediction.",
)

print(results_table.to_dataframe())

    tvt_split  true_label                              predicted_probability  \
id                                                                             
0        test           2  [0.3062620171344619, 0.4118762178000316, 0.209...   
1       train           2  [0.022162796962580895, 0.029600696160931746, 0...   
2       train           2  [0.24781230936966467, 0.33159672522808925, 0.3...   
3       train           2  [0.0010764913933024565, 0.0014851515814415539,...   
4       train           2  [0.02275027657533403, 0.030764134673612387, 0....   
..        ...         ...                                                ...   
553      test           0  [0.30415485770991196, 0.4005894872411785, 0.19...   
554     train           0  [0.3340095255181371, 0.4349756760531751, 0.172...   
555     train           1  [0.30222976392447676, 0.3954133144575274, 0.18...   
556      test           1  [0.32269639174950426, 0.4098561444755049, 0.17...   
557      test           0  [0.3079885124

/Users/rly/mambaforge/envs/hdmf-ai/lib/python3.11/site-packages/hdmf/container.py:213: UserWarning: EnumData is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))
/Users/rly/mambaforge/envs/hdmf-ai/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Column 'samples' is predefined in ResultsTable with table=True which does not match the entered table argument. The predefined table spec will be ignored. Please ensure the new column complies with the spec. This will raise an error in a future version of HDMF.
  return func(args[0], **pargs)
/Users/rly/mambaforge/envs/hdmf-ai/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Column 'samples' is predefined in ResultsTable with class=<class 'hdmf.common.table.VectorData'> which does not match the entered col_cls argument. The predefined class spec will be ignored. Please ensure the new column complies with the spec. This will raise an error i

In [4]:
# access information about the first sample using the ResultsTable link to the input data
results_table["samples"][0]

,spike_times,unit_name,presence_ratio_standard_deviation,contamination,noise_cutoff,mean_relative_depth,sliding_refractory_period_violation,cosmos_location,maximum_amplitude,maximum_amplitude_channel,...,median_amplitude,drift,minimum_amplitude,spike_count,firing_rate,missed_spikes_estimate,standard_deviation_amplitude,allen_location,spike_amplitudes,presence_ratio
id,,,,,,,,,,,,,,,,,,,,,
0,"[0.019766666666666665, 0.028533333333333334, 0...",12,5.258554,0.083082,136.186874,80.0,0.0,MB,0.000175,6.0,...,0.000087,68480.697449,0.000082,4319.0,0.995299,0.5,0.725555,RN,"[8.288184885804525e-05, 0.00011442759068298115...",0.997701


In [5]:
# add the results table to the in-memory NWB file
nwbfile.add_analysis(results_table)

,tvt_split,true_label,predicted_probability,predicted_class,samples,log_probability
id,,,,,,
0,2,2,"[0.3062620171344619, 0.4118762178000316, 0.20988046099736338, 0.04890714752571586, 0.02307415654242729]",1,0,"[-1.1833142782703214, -0.8870324170073532, -1.5612171436853666, -3.017831727012486, -3.7690420521355175]"
1,0,2,"[0.022162796962580895, 0.029600696160931746, 0.9136374685313745, 0.03459903834511291, 5.037640062909857e-22]",2,1,"[-3.8093402083447496, -3.5199573989782915, -0.09032142895619419, -3.3639293907985515, -49.039934314931145]"
2,0,2,"[0.24781230936966467, 0.33159672522808925, 0.37279662191900254, 0.04779317482624241, 1.1686570012080219e-06]",2,2,"[-1.3950836364055648, -1.103835731438974, -0.9867222575878216, -3.0408724357415013, -13.659655330668379]"
3,0,2,"[0.0010764913933024565, 0.0014851515814415539, 0.9850122152347303, 0.012426141790525586, 1.1759901715724957e-36]",2,3,"[-6.834048236266172, -6.51223843689196, -0.015101236633409822, -4.387952816611791, -82.73095285585052]"


In [6]:
# add information about the model used to generate the results table
from hdmf.common import HERD

# annotate the model with a DOI using HDMF HERD
herd = HERD()
herd.add_ref(
    file=nwbfile,
    container=results_table,
    attribute="pre_trained_model",
    key='Bloom v1.3',
    entity_id='doi:10.57967/hf/0003',
    entity_uri='https://doi.org/10.57967/hf/0003'
)
herd.to_zip(path='./HERD.zip')

# remove the acquisition data from the NWB file to reduce file size
for x in list(nwbfile.acquisition.keys()):
    nwbfile.acquisition.pop(x)

# export the results table to a new NWB file
with NWBHDF5IO("results.nwb", "w") as export_io:
    export_io.export(src_io=io, nwbfile=nwbfile)

/Users/rly/mambaforge/envs/hdmf-ai/lib/python3.11/site-packages/hdmf/container.py:213: UserWarning: HERD is experimental -- it may be removed in the future and is not guaranteed to maintain backward compatibility
  warn(_exp_warn_msg(cls))
/Users/rly/mambaforge/envs/hdmf-ai/lib/python3.11/site-packages/hdmf/build/objectmapper.py:260: DtypeConversionWarning: Spec 'ResultsTable/VectorData': Value with data type int64 is being converted to data type uint64 (min specification: uint8).
  warnings.warn(full_warning_msg, DtypeConversionWarning)
